In [1]:
""" use kernel tensorflow_p36 - NOT TF 2"""
!pip install --upgrade tensorflow==1.8.0

  Using cached tensorflow-1.8.0-cp36-cp36m-manylinux1_x86_64.whl (49.1 MB)
  Using cached tensorboard-1.8.0-py3-none-any.whl (3.1 MB)
  Using cached bleach-1.5.0-py2.py3-none-any.whl (17 kB)
Processing /home/ec2-user/.cache/pip/wheels/90/1c/cb/a87fd097ff74648ecc468a703001f6c7c86d8a71d459e65c98/html5lib-0.9999999-py3-none-any.whl
ERROR: tensorflow-serving-api 1.15.0 has requirement tensorflow~=1.15.0, but you'll have tensorflow 1.8.0 which is incompatible.
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: bleach
    Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Attempting uninstall: tensorflow
    Found existing

In [2]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf 

from sklearn.model_selection import train_test_split

EPOCHS = 20
IMG_WIDTH = 224
IMG_HEIGHT = 224
NUM_CATEGORIES = 2
TEST_SIZE = 0.2

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.i

In [3]:
def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = []
    labels=[]
    counter = 0
    for labelname in os.listdir(data_dir):
        path= data_dir + "/" + labelname 
        for pic in os.listdir(path):
            path2= data_dir + "/" + labelname + "/" + pic
            img= cv2.imread(path2)
            #print ("path- ", path)
            #print ("path2- ", path2)
            re_img=cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
            images.append(re_img)
            labels.append(labelname)
            counter= counter+1
    print(counter) 
    return (images,labels)


def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
        """
# Create a convolutional neural network
    model = tf.keras.models.Sequential([

         # Convolutional layer. Learn 32 filters using a 3x3 kernel
         tf.keras.layers.Conv2D(
             32, (3, 3), activation="relu", padding = "same", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
         ),


         # Max-pooling layer, using 3x3 pool size
         tf.keras.layers.MaxPooling2D(pool_size=(4, 4)),
         tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv2D(
             64, (3, 3), activation="relu", padding = "same", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
         ),


         # Max-pooling layer, using 3x3 pool size
         tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
         tf.keras.layers.Dropout(0.2),
         

         # Flatten units
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.2),
        
        

         # Add a hidden layer with dropout
         tf.keras.layers.Dense(64, activation="relu"),
         tf.keras.layers.Dropout(0.1),
        
         # Add a hidden layer with dropout
         tf.keras.layers.Dense(64, activation="relu"),
         tf.keras.layers.Dropout(0.1),
         
         # Flatten units
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.1),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(32, activation="relu"),
         tf.keras.layers.Dropout(0.1),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(32, activation="relu"),
         tf.keras.layers.Dropout(0.1),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(32, activation="relu"),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(64, activation="relu"),
         tf.keras.layers.Dropout(0.1),
         
         # Flatten units
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.1),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(32, activation="relu"),
         tf.keras.layers.Dropout(0.1),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(32, activation="relu"),
         tf.keras.layers.Dropout(0.1),
        
        

        # Add a hidden layer with dropout
         tf.keras.layers.Dense(16, activation="relu"),
         tf.keras.layers.Dropout(0.1),
         
         # Add an output layer with number of categories
         tf.keras.layers.Dense(2, activation="softmax")
     ])

# Train neural network
    model.compile(
         optimizer="adam",
         loss="categorical_crossentropy",
         metrics=["accuracy"]
   )

    return model



In [4]:
# Get image arrays and labels for all image files
    images, labels = load_data("/home/ec2-user/SageMaker/BrainScans/brain")

    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE)


253


In [5]:
model = get_model()


In [6]:
import time

start = time.time()

# Fit model on training data
model_history = model.fit(x_train, y_train, epochs=EPOCHS)

end = time.time()
print(end - start)

Epoch 1/20
202/202 [==============================] - 6s 28ms/step - loss: 5.1365 - acc: 0.6089
Epoch 2/20
202/202 [==============================] - 5s 23ms/step - loss: 3.3433 - acc: 0.5693
Epoch 3/20
202/202 [==============================] - 5s 23ms/step - loss: 0.9140 - acc: 0.5792
Epoch 4/20
202/202 [==============================] - 5s 23ms/step - loss: 0.7006 - acc: 0.4851
Epoch 5/20
202/202 [==============================] - 5s 24ms/step - loss: 0.6982 - acc: 0.5000
Epoch 6/20
202/202 [==============================] - 5s 23ms/step - loss: 0.6940 - acc: 0.5347
Epoch 7/20
202/202 [==============================] - 5s 23ms/step - loss: 0.6949 - acc: 0.5099
Epoch 8/20
202/202 [==============================] - 5s 23ms/step - loss: 0.6477 - acc: 0.6238
Epoch 9/20
202/202 [==============================] - 5s 23ms/step - loss: 0.6259 - acc: 0.6634
Epoch 10/20
202/202 [==============================] - 5s 23ms/step - loss: 0.5906 - acc: 0.6584
Epoch 11/20
202/202 [==================

In [8]:
   # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=1)

51/51 [==============================] - 0s 9ms/step


[0.6589848258916069, 0.7843137278276331]

In [9]:
#model.summary()

In [8]:
model.save("model-latest1")